In [3]:
cd /home

/home


In [10]:
import tensorflow as tf
import pandas as pd 

import matplotlib.pyplot as plt 
import os 
import tomli

In [4]:
config_files = './presentation/pipeline/config/alcock/a/'

In [16]:
for file in os.listdir(config_files):
    
    current = os.path.join(config_files, file)
    with open(current, mode="rb") as fp:
        config = tomli.load(fp)
           
    metrics = pd.read_csv(os.path.join(config['finetuning']['exp_path'], 'metrics.csv'))
    metrics = pd.read_csv(os.path.join(config['classification']['exp_path'], 'metrics.csv'))
    metrics['model'] = ['mlp_att', 'lstm_att', 'lstm']
    print(metrics)


       loss       acc     model
0  1.668646  0.271933   mlp_att
1  1.848123  0.193624  lstm_att
2  1.236768  0.488496      lstm
       loss       acc     model
0  1.600582  0.296050   mlp_att
1  1.847118  0.179903  lstm_att
2  0.826095  0.660430      lstm
       loss       acc     model
0  1.715071  0.215067   mlp_att
1  1.833275  0.191498  lstm_att
2  1.175764  0.510802      lstm
       loss       acc     model
0  1.690782  0.259680   mlp_att
1  1.866841  0.180976  lstm_att
2  1.405258  0.444094      lstm
       loss       acc     model
0  1.737247  0.226611   mlp_att
1  1.871541  0.178240  lstm_att
2  1.396447  0.435759      lstm
       loss       acc     model
0  1.577365  0.296236   mlp_att
1  1.817002  0.193646  lstm_att
2  0.856925  0.652502      lstm
       loss       acc     model
0  1.713801  0.230676   mlp_att
1  1.890144  0.185620  lstm_att
2  1.481464  0.403953      lstm
       loss       acc     model
0  1.768857  0.215067   mlp_att
1  1.865382  0.167859  lstm_att
2  1.601

In [ ]:
./results/finetuning/alcock/a/alcock_100_f1/metrics.csv